In [ ]:
#!/usr/bin/env python3
"""
run_dnn_slope_vs_yields.py  ──  slope+yields vs yields-only 비교
=============================================================

* SLOPE+YIELDS : static 듀얼-브랜치 DNN_DUAL  
* YIELDS-only  : 단일-브랜치 DNN  

※ OOS-R² 와 MSE 모두 Machine.xxxOOS() 메서드로 가져옵니다.
"""
import os, sys, argparse, warnings
import pandas as pd
warnings.filterwarnings("ignore")

from rolling_framework import Machine

# ─────────────────────────── USER CONFIG ───────────────────────────
DATA_ROOT     = "./data/19712023"
OUTPUT_ROOT   = "./output"
OUT_DEFAULT   = "results_slope_vs_yields.csv"

BURN_START    = "197108"
BURN_END      = "199001"
PERIOD_START  = "197108"
PERIOD_END    = "202312"
HORIZON       = 12  # months ahead

MATURITIES    = ["xr_2","xr_3","xr_5","xr_7","xr_10"]

# 하이퍼-파라미터 그리드
param_grid_base = {
    "dnn__module__hidden"          : [(16,)],
    "dnn__module__dropout"         : [0.2],
    "dnn__optimizer__lr"           : [1e-3],
    "dnn__optimizer__weight_decay" : [1e-4],
}
param_grid_dual = {
    # branch2 (yields) 구조
    "dnn__module__hidden2"         : [(16,)],
    "dnn__module__drop2"           : [0.2],
    # 합치는 방식
    "dnn__module__merge"           : ["gate"],
    # optimizer
    "dnn__optimizer__lr"           : [1e-3],
    "dnn__optimizer__weight_decay" : [1e-4],
    # branch별 lr (SafeNetDualLR 사용 시)
    "dnn__lr_br1"                   : [5e-4],
    "dnn__lr_br2"                   : [1e-3],
    "dnn__lr_head"                  : [1e-3],
}

def main():
    # Jupyter/VSCode 런처 플래그 무시
    p = argparse.ArgumentParser(add_help=False)
    p.add_argument("--out", default=None, help="출력 CSV 경로")
    args, _ = p.parse_known_args()

    os.makedirs(OUTPUT_ROOT, exist_ok=True)
    out_csv = args.out or os.path.join(OUTPUT_ROOT, OUT_DEFAULT)

    # 1) 데이터 로드
    try:
        y  = pd.read_csv(os.path.join(DATA_ROOT, "exrets_19712023.csv"),
                         index_col="Time")
        yl = pd.read_csv(os.path.join(DATA_ROOT, "yl_all_19712023.csv"),
                         index_col="Time")
        ls = pd.read_csv(os.path.join(DATA_ROOT, "lsc_19712023.csv"),
                         index_col="Time")
    except FileNotFoundError as e:
        sys.exit(f"[ERROR] CSV not found → {e.filename}")

    # 평가할 만기만 선택
    cols = [c for c in MATURITIES if c in y.columns]
    if not cols:
        sys.exit(f"[ERROR] None of {MATURITIES} in exrets")
    y = y[cols]

    rows = []

    # A) SLOPE+YIELDS (static 듀얼-브랜치)
    X_sly = pd.concat([ls[["slope"]], yl], axis=1)
    m_sly = Machine(
        X_sly, y, "DNN_DUAL",
        option           = {
            "slope": "slope",
            "grp2" : yl.columns.tolist(),
            "merge": "gate"
        },
        params_grid      = param_grid_dual,
        burn_in_start    = BURN_START,
        burn_in_end      = BURN_END,
        period           = [PERIOD_START, PERIOD_END],
        forecast_horizon = HORIZON,
    )
    m_sly.training()

    r2_sly  = m_sly.R2OOS()    # Series: 각 만기별 OOS-R²
    mse_sly = m_sly.MSEOOS()   # Series: 각 만기별 OOS-MSE

    for mty in cols:
        rows.append({
            "set"      : "SLOPE+YIELDS",
            "maturity" : mty,
            "model"    : "DNN_DUAL_SLP",
            "R2_OOS"   : float(r2_sly [mty]),
            "MSE"      : float(mse_sly[mty]),
        })

    # B) YIELDS-only (단일-브랜치)
    X_y = yl.copy()
    m_y = Machine(
        X_y, y, "DNN",
        option           = None,
        params_grid      = param_grid_base,
        burn_in_start    = BURN_START,
        burn_in_end      = BURN_END,
        period           = [PERIOD_START, PERIOD_END],
        forecast_horizon = HORIZON,
    )
    m_y.training()

    r2_y  = m_y.R2OOS()
    mse_y = m_y.MSEOOS()

    for mty in cols:
        rows.append({
            "set"      : "YIELDS",
            "maturity" : mty,
            "model"    : "DNN",
            "R2_OOS"   : float(r2_y [mty]),
            "MSE"      : float(mse_y[mty]),
        })

    # 4) 결과 저장
    pd.DataFrame(rows) \
      .sort_values(["set","maturity","model"]) \
      .to_csv(out_csv, index=False)

    print(f"★ Saved results → {out_csv}")

if __name__ == "__main__":
    main()

DNN_DUAL rolling:   4%|▍         | 20/520 [03:47<1:34:56, 11.39s/it]
